In [1]:
setwd('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1. InVitroWildfire EVProteomics/P1.2. Original Data')
Output = ('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1. InVitroWildfire EVProteomics/P1.3. Analyses/P1.3.1. PDF Data Extraction/Output')
cur_date = "011122"

library(tidyverse)
# library(reshape2)
library(pdftools)
library(tm)

#only grabs files ending in "pdf"
all_files = list.files(pattern = "pdf$")
head(all_files)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Using poppler version 20.12.1

Loading required package: NLP


Attaching package: ‘NLP’


The following object is masked from ‘package:ggplot2’:

    annotate




[1] "20211018_0002_9.2.1_84_200_size.pdf"  
[2] "20211018_0003_9.2.1_95_200_size.pdf"  
[3] "20211018_0004_9.2.1_101_200_size.pdf" 
[4] "20211018_0005_9.2.1_109_2000_size.pdf"
[5] "20211018_0006_9.2.1_124_2000_size.pdf"
[6] "20211018_0007_9.2.1_125_2000_size.pdf"

In [12]:
c = "Original Concentration"
length(c)

[1] 1

In [14]:
for (i in 1:length(all_files)){
    #pdftools extracts text from each file
    #this gives an error, but we still get each line of text that's now stored in the vector
    extracted_text[i] = pdf_text(all_files[i]) %>%
    #text is currently class 'list', but converted to character to make parsing possible
    as.character %>%
    strsplit(split = "\n")
    
    one_line = extracted_text[i]
    #print(one_line)
    
    #delete later: just wanted to see first pdf file
    if (i == 1){
        for (j in 1:length(one_line)){
            print(j)
            #if (j:j+24 == "Original Concentration"){
                
            #}
        }
    }
}

PDF error: Invalid least number of objects reading page offset hints table



[1] 1


PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: I

DELETE BELOW

In [2]:
#pdftools extracts text from each file
#this gives an error, but we still get the text that's now stored in the vector
extracted_text_files = lapply(all_files, pdf_text) %>%
    #text is currently class 'list', but converted to character to make parsing possible
    as.character()
extracted_text_files[1]

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: I

[1] "                                                                                                      Electrophoresis & Brownian Motion\n                                                                                                                Video Analysis\n                                                                                                         Laser Scattering Microscopy\n                                           www.particle-metrix.de\n\n                                                                                       Operator (Report): sop_kabanovlab.inst\n                                                                                       Video Operator: sop_kabanovlab.inst\n\n\n\n\n Sample Parameters                                                                     Result (sizes in nm)\n Sample Name: 9.2.1_84_200                                                                                Number      Concentration   Volume\n Comment: Sample Remarks0:                                                             Median (X50)        123.3       123.3      1907.5\n Sample Remarks1:                                                                      Span                180.7       180.3      619.5\n Sample Remarks2:\n Electrolyte:                                                                          Concentration:                 2.4E+7 Particles / mL\n Temperature: 23.04 °C sensed                                                          Dilution Factor:                     200\n pH 7.0 entered                                                                        Original Concentration:        4.7E+9 Particles / mL\n Conductivity: 15000.00 µS/cm sensed\n\n                                                                                       Quality\n                                                                                       Average Counted Particles per Frame: 52\n Measurement Parameters                                                                Number of Traced Particles: 234\n Cell S/N: ZNTA\n\n Measurement Mode: Size Distribution 1 Cycle                                           Analysis Parameters\n 11 Positions, 2 Removed for Analysis                                                  Max Area: 10000, Min Area: 5, Min Brightness: 20\n 8.0E+6                                                                                2.4E+8\n                                                                                       2.2E+8\n 7.0E+6\n                                                                                        2E+8\n 6.0E+6                                                                                1.8E+8\n                                                                                       1.6E+8\n 5.0E+6\n                                                                                       1.4E+8\n\n 4.0E+6                                                                                1.2E+8\n                                                                                        1E+8\n 3.0E+6\n                                                                                        8E+7\n\n 2.0E+6                                                                                 6E+7\n                                                                                        4E+7\n 1.0E+6\n                                                                                        2E+7\n\n 0.0E+0                                                                                     0\n          0   100   200     300   400 500 600      700     800   900   1000                     0   100   200   300   400  500     600   700    800   900   1000\n                                   Diameter / nm                                                                       Diameter / nm\n\n\n\n\nPeak Analysis (Concentration)\nDiameter / nm             Particles/mL       FWHM / nm    

In [5]:
b = extracted_text_files[1] %>%
    strsplit(split = "\n")
length(b)
b

[1] 1

[[1]]
 [1] "                                                                                                      Electrophoresis & Brownian Motion"                                   
 [2] "                                                                                                                Video Analysis"                                            
 [3] "                                                                                                         Laser Scattering Microscopy"                                      
 [4] "                                           www.particle-metrix.de"                                                                                                         
 [5] ""                                                                                                                                                                          
 [6] "                                                                                       Operator (Report): sop_kabanovlab.inst"                                             
 [7] "                                                                                       Video Operator: sop_kabanovlab.inst"                                                
 [8] ""                                                                                                                                                                          
 [9] ""                                                                                                                                                                          
[10] ""                                                                                                                                                                          
[11] ""                                                                                                                                                                          
[12] " Sample Parameters                                                                     Result (sizes in nm)"                                                               
[13] " Sample Name: 9.2.1_84_200                                                                                Number      Concentration   Volume"                              
[14] " Comment: Sample Remarks0:                                                             Median (X50)        123.3       123.3      1907.5"                                  
[15] " Sample Remarks1:                                                                      Span                180.7       180.3      619.5"                                   
[16] " Sample Remarks2:"                                                                                                                                                         
[17] " Electrolyte:                                                                          Concentration:                 2.4E+7 Particles / mL"                               
[18] " Temperature: 23.04 °C sensed                                                          Dilution Factor:                     200"                                           
[19] " pH 7.0 entered                                                                        Original Concentration:        4.7E+9 Particles / mL"                               
[20] " Conductivity: 15000.00 µS/cm sensed"                                                                                                                                      
[21] ""                                                                                                                                                                          
[22] "                                                                                       Quality"                                                                            
[23] "                                                                        

In [3]:
a = extracted_text_files[1]
class(a)
length(a)

[1] "character"

[1] 1

In [6]:
#however this text still isn't readable by R, so a corpus (database) for text is now created
corp = Corpus(URISource(all_files),
               readerControl = list(reader = readPDF))
head(corp)

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: I

<<VCorpus>>
Metadata:  corpus specific: 0, document level (indexed): 0
Content:  documents: 6

# Data we want to extract:

1. Pdf file name as the sample identifier
2. Original Concentration
3. Some of the ‘X Values’ table listed on the bottom left, specifically the: X10 Concentration, X50 Concentration, X90 Concentration, and Mean Concentration, and StdDev Concentration

In [4]:
#getting OG concentration
for (i in 1:length(extracted_text_files[1])){
#     if (extracted_text_files[1][i] == 'n'){
#         print(i)
#     }
    print(i)
}

[1] 1


In [17]:
length(extracted_text_files[1])

[1] 1